In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table

from utility.pyomo_preprocessing import extract_optimization_results, linear_interpolation_for_bound, arange_float, linear_interpolation_using_cols 
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings
from utility.general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

COLORS = px.colors.qualitative.Plotly



In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [8]:

from utility.general_function import duckdb_to_dict


results = []
for file in os.listdir(output_file_names["baseline"]):
    if file.endswith(".duckdb"):
        result = duckdb_to_dict(os.path.join(output_file_names["baseline"], file))
        results.append(result)
for result in results:
    print(result["income_result"].to_pandas().to_string())

Read tables from 2020_year_results.duckdb file: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.44it/s]

   turbine_factor  first_stage  model_second_stage_0  model_second_stage_1  model_second_stage_2  model_second_stage_3  model_second_stage_4
0             0.7        2.818                 3.326                 3.333                 3.332                 3.320                 3.320
1             0.8        2.891                 3.348                 3.354                 3.351                 3.338                 3.338
2             0.9        2.950                 3.362                 3.353                 3.357                 3.343                 3.343
   turbine_factor  first_stage  model_second_stage_0  model_second_stage_1  model_second_stage_2  model_second_stage_3  model_second_stage_4
0             0.7        7.930                 9.005                 9.067                 9.064                 9.066                 9.081
1             0.8        8.215                 9.107                 9.181                 9.170                 9.163                 9.156
2            

In [13]:
results[0]["turbine_factor_0_7_model_second_stage_0"]["spilled_volume_0"].sum()

-2.0816458011559016e-05

In [9]:
for result in results:
    print(result["log_book"].to_pandas().to_string())

          sim_name  sim_nb   lower_bound   upper_bound
0   second_stage_0     NaN           NaN           NaN
1   second_stage_1    56.0  -4036.109603  -3891.539273
2   second_stage_2    41.0 -14188.156318 -14060.857422
3   second_stage_2    56.0  -3854.903499  -3657.093454
4   second_stage_3    41.0 -16151.363771 -15864.320570
5   second_stage_3    55.0 -21474.221359 -21234.189542
6   second_stage_3    56.0  -2283.794064  -2097.766655
7   second_stage_3    87.0  32871.195013  32987.651724
8   second_stage_3    91.0 -34624.398207 -34106.888259
9   second_stage_4    41.0 -16151.376013 -15864.062535
10  second_stage_4    55.0 -21474.234206 -21232.119175
11  second_stage_4    56.0  -2283.806918  -2097.192983
12  second_stage_4    87.0  32871.128419  32987.654267
13  second_stage_4    91.0 -34624.379186 -34107.204220
14  second_stage_0     NaN           NaN           NaN
15  second_stage_1     NaN           NaN           NaN
16  second_stage_2     NaN           NaN           NaN
17  second

In [4]:
import duckdb
file_path = ".cache/output/baseline/2020_year_results.duckdb"
# with duckdb.connect(database=file_path) as con:
duckdb_to_dict(file_path)

Read tables from 2020_year_results.duckdb file: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.61it/s]


{'first_stage': shape: (366, 9)
 ┌─────┬────────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
 │ T   ┆ timestamp  ┆ market_pri ┆ max_market ┆ … ┆ basin_volu ┆ basin_volu ┆ income    ┆ hydro_0   │
 │ --- ┆ ---        ┆ ce         ┆ _price     ┆   ┆ me_0       ┆ me_1       ┆ ---       ┆ ---       │
 │ u32 ┆ datetime[μ ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ f64       ┆ struct[4] │
 │     ┆ s, Europe/ ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64        ┆           ┆           │
 │     ┆ Zurich]    ┆            ┆            ┆   ┆            ┆            ┆           ┆           │
 ╞═════╪════════════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
 │ 0   ┆ 2020-01-01 ┆ 31.815417  ┆ 41.14      ┆ … ┆ 0.811261   ┆ 0.0        ┆ 0.0       ┆ {0.0,0.0, │
 │     ┆ 01:00:00   ┆            ┆            ┆   ┆            ┆            ┆           ┆ 0.0,0.0}  │
 │     ┆ CET        ┆            ┆            ┆   

In [5]:
results

[{'first_stage': shape: (365, 9)
  ┌─────┬────────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
  │ T   ┆ timestamp  ┆ market_pri ┆ max_market ┆ … ┆ basin_volu ┆ basin_volu ┆ income    ┆ hydro_0   │
  │ --- ┆ ---        ┆ ce         ┆ _price     ┆   ┆ me_0       ┆ me_1       ┆ ---       ┆ ---       │
  │ u32 ┆ datetime[μ ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ f64       ┆ struct[4] │
  │     ┆ s, Europe/ ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64        ┆           ┆           │
  │     ┆ Zurich]    ┆            ┆            ┆   ┆            ┆            ┆           ┆           │
  ╞═════╪════════════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
  │ 0   ┆ 2023-01-01 ┆ 6.565417   ┆ 35.15      ┆ … ┆ 0.811261   ┆ 0.0        ┆ 0.0       ┆ {0.0,0.0, │
  │     ┆ 01:00:00   ┆            ┆            ┆   ┆            ┆            ┆           ┆ 0.0,0.0}  │
  │     ┆ CET        ┆            ┆     

In [6]:
results_2020["income_result"]

NameError: name 'results_2020' is not defined